In [1]:
### Install required packages here

### Prepare mean index for each bounds

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the CSV file
df = pd.read_csv(r"./csv/Bounds_HAI.csv")

In [3]:
df.head()

,AGG_DAY_PERIOD,BOUNDS,CENTROID_LAT,CENTROID_LON,MEAN_ACTIVITY_INDEX_TOTAL
0,2020-01-20,"(5.873565673828125, 51.04053042130972, 5.87493...",51.040962,5.874252,0.018629
1,2020-01-20,"(5.87493896484375, 51.02844005602988, 5.876312...",51.028872,5.875626,0.164530
2,2020-01-20,"(5.876312255859375, 51.03103111414404, 5.87768...",51.031463,5.876999,0.203132
3,2020-01-20,"(5.876312255859375, 51.04053042130972, 5.87768...",51.040962,5.876999,0.233522
4,2020-01-20,"(5.876312255859375, 51.04139389812637, 5.87768...",51.041826,5.876999,0.009507


In [4]:
len(df)
df.nunique()

AGG_DAY_PERIOD                   347
BOUNDS                       3005863
CENTROID_LAT                    3072
CENTROID_LON                    2896
MEAN_ACTIVITY_INDEX_TOTAL     484241
dtype: int64